### AITech — Uczenie maszynowe — laboratoria
# 4. Regresja logistyczna

Do wykonania zadań wykorzystaj wiedzę z wykładów *3. Regresja logistyczna* i *4. Metody ewaluacji*.

## Zadanie 4 (6 punktów)

Plik *gratkapl-centrenrm.csv* zawiera dane wzbogacone o nową cechę *Centre*, przyjmującą wartość 1, jeżeli lokalizacja mieszkania to centrum, i 0 w przeciwnym przypadku.

Zamiast przywidywania ceny mieszkania, spróbuj przewidzieć lokalizację mieszkania: *centrum* vs. *nie-centrum* za pomocą regresji logistcznej.

 1. Podziel dane trenujące na nowe dane treningowe (pierwsze 4/5 danych) i testowe (ostatnie 1/5 danych)
 1. Stwórz dwuklasowy model regresji logistycznej, korzystając z cech liczbowych *Price,Rooms,SqrMeters,Floor*. Warto usunąć obserwacje odstające i dokonać normalizacji danych.
 1. Podaj procent poprawnych wyników (tzw. skuteczność, *accuracy*) dla zestawu testowego.
 
Otrzymaną wartość porównaj z wynikiem uzyskiwanym przez klasyfikator losowy (czyli taki, który przyporządkowuje losową wartość 0 lub 1 z prawdopodobieństem 1/2).

**Uwaga**: Cecha *Centre* została wygenerowana automatycznie na podstawie opisów lokalizacji mieszkań i okazuje się, że jest słabo skorelowana z cechami liczbowymi, dlatego proszę się nie przejmować, jeżeli wartości funkcji regresji logistycznej i przewidywania modelu będą nieco dziwne.

In [9]:
# Przydatne importy

import numpy as np
import matplotlib
import matplotlib.pyplot as pl
import pandas as pd
import ipywidgets as widgets
import random

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from IPython.display import display, Math, Latex, Markdown


In [2]:
# wczytanie danych i podzial
df = pd.read_csv("gratkapl-centrenrm.csv", usecols=["Price", "Rooms", "SqrMeters", "Floor", "Centre"])
df_split = np.array_split(df, 5)
df_test = df_split[-1]
df_train = pd.concat(df_split[:4])
print(df_train)
print(df_test)

       Price  Rooms  SqrMeters  Floor  Centre
0     386064      2      46.00      3       1
1     280000      2      41.00      5       0
2     329000      2      50.50      1       0
3     225000      2      43.00     10       1
4     280000      2      69.00      3       0
...      ...    ...        ...    ...     ...
1610  261000      2      44.30      3       1
1611  605520      3      84.00      2       1
1612  380000      2      41.11      1       0
1613  209000      1      31.00      5       1
1614  217000      2      56.00      3       1

[1615 rows x 5 columns]
        Price  Rooms  SqrMeters  Floor  Centre
1615   690000      2     121.00      1       1
1616   197550      2      49.40      2       1
1617   330000      4      74.50      1       0
1618  1140000      6     192.40      1       0
1619   194000      1      36.10      1       0
...       ...    ...        ...    ...     ...
2013   277000      2      46.10      3       0
2014   371000      3      47.00      2       0


In [3]:
# przeksztalcenie na macierze

m, n_plus_1 = df_train.values.shape
n = n_plus_1 - 1
m2, n_plus_1_2= df_test.values.shape
n2 = n_plus_1_2 - 1
Xn = df_train.values[:, 0:n].reshape(m, n)
Xn2 = df_test.values[:, 0:n2].reshape(m2, n2)

XTrain = np.matrix(np.concatenate((np.ones((m, 1)), Xn), axis=1)).reshape(m, n_plus_1)
ytrain = np.matrix(df_train.values[:, n]).reshape(m, 1)

XTest = np.matrix(np.concatenate((np.ones((m2, 1)), Xn2), axis=1)).reshape(m2, n_plus_1_2)
ytest = np.matrix(df_test.values[:, n2]).reshape(m2, 1)
# XMx4 = np.matrix(np.concatenate((np.ones((m, 1)), Xn), axis=1)).reshape(m, n_plus_1)
# yMx4 = np.matrix(df.values[:, n]).reshape(m, 1)

# print(XMx4[:6])
# print(yMx4[:6])
# print(XTrain)
# print(ytrain)

# print(XTest)
# print(ytest)


In [4]:
# normalizacja dnaych

def show_mins_and_maxs(XMx):
    mins = np.amin(XMx, axis=0).tolist()[0]  # wartości minimalne
    maxs = np.amax(XMx, axis=0).tolist()[0]  # wartości maksymalne
    for i, (xmin, xmax) in enumerate(zip(mins, maxs)):
        display(Math(
            r'${:.2F} \leq x_{} \leq {:.2F}$'.format(xmin, i, xmax)))
        

XTrain_norm = (XTrain - np.mean(XTrain, axis=0)) / np.amax(XTrain, axis=0)
XTest_norm = (XTest - np.mean(XTest, axis=0)) / np.amax(XTest, axis=0)
print(XTrain_norm)
show_mins_and_maxs(XTrain_norm)
show_mins_and_maxs(XTest_norm)

[[ 0.00000000e+00  2.56864539e-05 -7.64705882e-02 -4.60265507e-02
   3.79256966e-03]
 [ 0.00000000e+00 -2.88988950e-03 -7.64705882e-02 -6.54818036e-02
   1.28792570e-01]
 [ 0.00000000e+00 -1.54293651e-03 -7.64705882e-02 -2.85168231e-02
  -1.21207430e-01]
 ...
 [ 0.00000000e+00 -1.41005851e-04 -7.64705882e-02 -6.50537881e-02
  -1.21207430e-01]
 [ 0.00000000e+00 -4.84159689e-03 -2.43137255e-01 -1.04392309e-01
   1.28792570e-01]
 [ 0.00000000e+00 -4.62168620e-03 -7.64705882e-02 -7.11604486e-03
   3.79256966e-03]]


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [5]:
# regresja logistyczna

# Funkjca logistycza
thetaTemp = np.ones(5).reshape(5,1)


def logistic(x):
    return 1.0 / (1.0 + np.exp(-x))


# Funkcja regresji logistcznej
def h(theta, X):
    return 1.0/(1.0 + np.exp(-X * theta))


# Funkcja kosztu dla regresji logistycznej
def J(h, theta, X, y):
    m = len(y)
    h_val = h(theta, X)
    s1 = np.multiply(y, np.log(h_val))
    s2 = np.multiply((1 - y), np.log(1 - h_val))
    return -np.sum(s1 + s2, axis=0) / m


# Gradient dla regresji logistycznej
def dJ(h, theta, X, y):
    return 1.0 / len(y) * (X.T * (h(theta, X) - y))


# Metoda gradientu prostego dla regresji logistycznej
def GD(h, fJ, fdJ, theta, X, y, alpha=0.01, eps=10**-3, maxSteps=10000):
    errorCurr = fJ(h, theta, X, y)
    errors = [[errorCurr, theta]]
    while True:
        # oblicz nowe theta
        theta = theta - alpha * fdJ(h, theta, X, y)
        # raportuj poziom błędu
        errorCurr, errorPrev = fJ(h, theta, X, y), errorCurr
        # kryteria stopu
        if abs(errorPrev - errorCurr) <= eps:
            break
        if len(errors) > maxSteps:
            break
        errors.append([errorCurr, theta]) 
    return theta, errors


# Uruchomienie metody gradientu prostego dla regresji logistycznej
thetaBest, errors = GD(h, J, dJ, thetaTemp, XTrain_norm, ytrain, 
                       alpha=0.1, eps=10**-7, maxSteps=1000)
print("error =", errors[-1][0])
print("theta =", thetaBest)

error = [[0.69195052]]
theta = [[1.        ]
 [0.93331417]
 [0.18175774]
 [1.10451816]
 [0.48075158]]


In [12]:
# funkcja decyzyjna + skutecznosc modelu
def classifyBi(theta, X):
    prob = h(theta, X).item()
    return (1, prob) if prob > 0.5 else (0, prob)

print("theta =", thetaBest)
print("x0 =", XTest_norm[0])
print("h(x0) =", h(thetaBest, XTest_norm[0]).item())
print("c(x0) =", classifyBi(thetaBest, XTest_norm[0]), "\n")

acc = 0.0
for i, rest in enumerate(ytest):
    cls, prob = classifyBi(thetaBest, XTest_norm[i])
    if i < 10:
        display(Markdown(rf"Test value: ${int(ytest[i].item())}$ Predicted value: ${cls}$ -- prob: ${round(prob, 4)}$"))
#         print("Test value: ", int(ytest[i].item()), "Predicted value: ", cls, "-- prob:", round(prob, 4))
    acc += cls == ytest[i].item()

theta = [[1.        ]
 [0.93331417]
 [0.18175774]
 [1.10451816]
 [0.48075158]]
x0 = [[ 0.          0.33618676 -0.0624483   0.33779618 -0.12371499]]
h(x0) = 0.6493181112131285
c(x0) = (1, 0.6493181112131285) 



Test value: $1$ Predicted value: $1$ -- prob: $0.6493$

Test value: $1$ Predicted value: $0$ -- prob: $0.4591$

Test value: $0$ Predicted value: $1$ -- prob: $0.5287$

Test value: $0$ Predicted value: $1$ -- prob: $0.8199$

...

Test value: $0$ Predicted value: $0$ -- prob: $0.4699$

Test value: $0$ Predicted value: $1$ -- prob: $0.5013$

Test value: $0$ Predicted value: $0$ -- prob: $0.49$

Test value: $1$ Predicted value: $0$ -- prob: $0.4452$

Test value: $0$ Predicted value: $0$ -- prob: $0.4642$

In [19]:
# losowy klasyfikator + porownanie dokladnosci
random_acc = 0.0

for i, rest in enumerate(ytest):
    cls, prob = (random.randint(0, 1), 0.5)
    if i < 10:
        display(Markdown(rf"Test value: ${int(ytest[i].item())}$ Predicted value: ${cls}$ -- prob: ${round(prob, 4)}$"))
    random_acc += cls == ytest[i].item()

#print(f"Trained model accuracy: {acc / len(XTest_norm)}\nRandom classifier accuracy: {random_acc / len(XTest_norm)}")
display(Markdown(rf"### Trained model accuracy: ${acc / len(XTest_norm)}$<br> Random classifier accuracy: ${random_acc / len(XTest_norm)}$"))

Test value: $1$ Predicted value: $1$ -- prob: $0.5$

Test value: $1$ Predicted value: $0$ -- prob: $0.5$

Test value: $0$ Predicted value: $1$ -- prob: $0.5$

Test value: $0$ Predicted value: $1$ -- prob: $0.5$

Test value: $0$ Predicted value: $1$ -- prob: $0.5$

...

Test value: $0$ Predicted value: $1$ -- prob: $0.5$

Test value: $0$ Predicted value: $1$ -- prob: $0.5$

Test value: $1$ Predicted value: $1$ -- prob: $0.5$

Test value: $0$ Predicted value: $1$ -- prob: $0.5$

### Trained model accuracy: $0.5483870967741935$<br> Random classifier accuracy: $0.4913151364764268$